#### Importing the datasets and libraries

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [30]:
data_yelp = pd.read_csv('yelp_labelled.txt', sep='\t',header=None)
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [31]:
# Assign column names
columan_name = ['Review', 'Sentiment']
data_yelp.columns = columan_name

In [32]:
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [33]:
data_yelp.shape

(1000, 2)

In [34]:
data_amazon = pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)
data_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [35]:
data_amazon.columns = columan_name

In [36]:
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [37]:
data_imdb = pd.read_csv('imdb_labelled.txt',sep='\t',header=None)
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [38]:
data_imdb.columns = columan_name

In [39]:
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [40]:
data_imdb.shape

(748, 2)

In [41]:
data = pd.concat([data_yelp,data_amazon, data_imdb],ignore_index=True)
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [42]:
data.shape

(2748, 2)

#### check distribution of sentiments

In [43]:
data['Sentiment'].value_counts()

Sentiment
1    1386
0    1362
Name: count, dtype: int64

In [44]:
# check for null values
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [45]:
x = data['Review']
y = data['Sentiment']

#### Data Cleaning

In [46]:
import string

In [47]:
punct = string.punctuation

In [48]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [49]:
import spacy

In [50]:
from spacy.lang.en.stop_words import STOP_WORDS

In [51]:
stopwords = list(STOP_WORDS) # list of stopwords

#### creating a function for data cleaning

In [52]:
import spacy 

nlp = spacy.load('en_core_web_sm')

def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
  
  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [53]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")

['hello', 'beautiful', 'day', 'outside']

#### Vectorization Feature Engineering (TF-IDF)

In [54]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [55]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)
# tokenizer=text_data_cleaning, tokenization will be done according to this function

In [56]:
classifier = LinearSVC()

#### Train the model

##### Splitting the dataset into the Train and Test set

In [57]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [58]:
x_train.shape, x_test.shape
# 2198 samples in training dataset and 550 in test dataset

((2198,), (550,))

#### Fit the x_train and y_train

In [59]:
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])
# it will first do vectorization and then it will do classification

In [60]:
clf.fit(x_train, y_train)

C:\Users\User\anaconda3\envs\nlp_2\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\User\anaconda3\envs\nlp_2\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x000002B1C3333700>)),
                ('clf', LinearSVC())])

#### Predict the Test set results

In [61]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [62]:
y_pred = clf.predict(x_test)

In [63]:
# confusion_matrix
confusion_matrix(y_test, y_pred)

array([[201,  78],
       [ 50, 221]], dtype=int64)

In [64]:
# classification_report
print(classification_report(y_test, y_pred))
# we are getting almost 77% accuracy

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       279
           1       0.74      0.82      0.78       271

    accuracy                           0.77       550
   macro avg       0.77      0.77      0.77       550
weighted avg       0.77      0.77      0.77       550



In [65]:
accuracy_score(y_test, y_pred)
# 76% accuracy

0.7672727272727272

In [66]:
clf.predict(["Wow, I am learning Natural Language Processing in fun fashion!"])
# output is 1, that means review is positive

array([1], dtype=int64)

In [67]:
clf.predict(["It's hard to learn new things!"])
# output is 0, that means review is Negative

array([0], dtype=int64)